Осуществим предобработку данных с Твиттера, чтобы очищенные данные в дальнейшем
использовать для задачи классификации. Данный датасет содержит негативные (label = 1)
и нейтральные (label = 0) высказывания. Для работы объединим train_df и test_df.
Задания:
1. Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим
функцию:
* для того, чтобы найти все вхождения паттерна в тексте, необходимо
использовать re.findall(pattern, input_txt)
* для для замены @user на пробел, необходимо использовать re.sub()
2. Изменим регистр твитов на нижний с помощью .lower().
3. Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя
apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в
тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в
качестве ключа (сокращенного слова), то заменить ключ на значение (полную
версию слова).
4. Заменим сокращения на их полные формы, используя short_word_dict. Для этого
воспользуемся функцией, используемой в предыдущем пункте.
5. Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict.
Для этого воспользуемся функцией, используемой в предыдущем пункте.
6. Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'.
7. Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.
8. Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'.
9. Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if
len(w)>1]).
10. Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый
столбец 'tweet_token'.
11. Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец
'tweet_token_filtered' без стоп-слов.
12. Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим
столбец 'tweet_stemmed' после применения стемминга.
13. Применим лемматизацию к токенам с помощью
nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после
применения лемматизации.
14. Сохраним результат предобработки в pickle-файл.

In [1]:
import numpy as np
import re
from matplotlib import pyplot as plt
from collections import Counter
import nltk
from nltk.tokenize import casual_tokenize, RegexpTokenizer, TreebankWordTokenizer
from nltk.util import ngrams
import pandas as pd
import math

In [16]:
import numpy as np
import pandas as pd
# import regex as re
import html
import nltk
import pickle
from dicts import apostrophe_dict, emoticon_dict, short_word_dict
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [3]:
STOP_WORDS = set(nltk.corpus.stopwords.words("english"))
STEMMER = nltk.stem.PorterStemmer()
LEMMATIZER = nltk.stem.wordnet.WordNetLemmatizer()

# Data load
train_df = pd.read_csv('/content/drive/MyDrive/NLP/Less_01/train_tweets.csv')
test_df = pd.read_csv('/content/drive/MyDrive/NLP/Less_01/test_tweets.csv')
df = train_df.append(test_df, ignore_index = True, sort = False)

In [4]:
print(train_df.info())
print(test_df.info())
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      31962 non-null  int64 
 1   label   31962 non-null  int64 
 2   tweet   31962 non-null  object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17197 entries, 0 to 17196
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      17197 non-null  int64 
 1   tweet   17197 non-null  object
dtypes: int64(1), object(1)
memory usage: 268.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49159 entries, 0 to 49158
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      49159 non-null  int64  
 1   label   31962 non-null  float64
 2   tweet   49159 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage:

In [10]:
def replace_from_dict(text, source_dict):
    
    """Поиск по тексту, сопоставление токенов через dict, размещение по умолчанию, если не в dict"""
    
    return " ".join([source_dict.get(word, word) for word in text.split()])


def remove_onechar_tokens(text):
    
    """Поиск по тексту, удаление односимвольных токенов"""
    
    return ' '.join([w for w in text.split() if len(w)>1])


def filter_stop_words(tokens, stop_words=STOP_WORDS):
    
    """Удалить стоп-слова из токенов"""
    
    return [token for token in tokens if token not in stop_words]
    

def stem_tokens(tokens, stemmer=STEMMER):
    
    """Предварительная обработка стемминга"""
    
    return [stemmer.stem(token) for token in tokens]


def lemmatize_tokens(tokens, lemmatizer=LEMMATIZER):
    
    """Лемматизация"""
    
    return [lemmatizer.lemmatize(token) for token in tokens]
    

def preprocess(df,
               src_col='tweet',
               clean_col='clean_tweet',
               token_col='tweet_token',
               filter_col='tweet_token_filtered',
               stemmed_col='tweet_stemmed',
               lem_col='tweet_lemmatized'
              ):
  # 1. Удалить ссылки @user
  df[clean_col] = df[src_col].apply(lambda x: re.sub(r'@[\w]*','', x))
    
  # 2. Правильный регистр в нижнем регистре
  df[clean_col] = df[clean_col].str.lower()
    
  # 3. Изменить апострофы
  vfunc = np.vectorize(replace_from_dict)
  df[clean_col] = vfunc(df[clean_col], apostrophe_dict)
    
  # 4. Продлить короткие слова
  df[clean_col] = vfunc(df[clean_col], short_word_dict)
    
  # 5. Заменить смайлики
  df[clean_col] = vfunc(df[clean_col], emoticon_dict)
    
  # 6. Заменить знаки препинания на пробелы
  df[clean_col] = df[clean_col].apply(lambda x: re.sub(r'[^\w\s]','', x))
    
  # 7. Заменить специальные символы на пробелы
  df[clean_col] = df[clean_col].apply(lambda x: re.sub(r'[^a-zA-Z0-9]', ' ', x))
    
  # 8. Заменить цифры на пробелы
  df[clean_col] = df[clean_col].apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x))
    
  # 9. Отбросьте односимвольное слово
  vfunc = np.vectorize(remove_onechar_tokens)
  df[clean_col] = vfunc(df[clean_col])
    
  # 10. Токенизировать текст
  df[token_col] = df[clean_col].apply(lambda x:  nltk.tokenize.word_tokenize(x))
    
  # 11. Отфильтровать стоп-слова
  stop_words = set(nltk.corpus.stopwords.words("english"))
  df[filter_col] = df[token_col].apply(lambda x: filter_stop_words(x))
    
  # 12. Apply stemming
  df[stemmed_col] = df[filter_col].apply(lambda x: stem_tokens(x))
    
  # 13. Лемматизировать
  df[lem_col] = df[stemmed_col].apply(lambda x: lemmatize_tokens(x))
    
  return df

In [17]:
%%time

result = preprocess(df)

CPU times: user 22.3 s, sys: 173 ms, total: 22.5 s
Wall time: 22.7 s


In [18]:
result.head()

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunct, selfish, drag, kid, dysfunc..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thank, lyft, credit, use, caus, offer, wheelc..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesti]"
3,4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguid, societi, motiv]"


In [20]:
# 14. Save to_pickle
result.to_pickle("/content/drive/MyDrive/NLP/Less_01/df_processed.pkl")